# V0  代名詞代換  
1.

In [1]:
from ckipnlp.pipeline import CkipPipeline, CkipDocument
import pandas as pd
from itertools import chain
from tqdm import trange

# from google.cloud import bigquery
# from google.cloud.exceptions import NotFound
import pandas as pd

## 載檔

In [2]:
file_full_name = '../../label/political_news_label_0_to_99.csv'
# file_output_name = file_full_name.split(".csv")[0]

news_df = pd.read_csv(file_full_name)
# all_output_file = "./v0" + file_output_name + "_result.csv"
# opinion_after_cluster_file = "./v0" + file_output_name + "_all_opinions_after_cluster.csv"
all_output_file = "v1_Total_Opinion.csv"
opinion_after_cluster_file = "v1_cluster_opinions.csv"
topic_info_file = "v1_topic_info.csv"

In [3]:
news_df.head(1)

,id,article,category,company_id,reporter,timestamp,title,url_pattern,label,annotator,annotation_id,created_at,updated_at,lead_time
0,883,"針對行政院長蘇貞昌、國民黨立委鄭麗文12日在立法院發生激烈爭執,兼任民進黨主席的總統蔡英文1...",政治,3,陶本和,1634110500,蔡英文發聲挺蘇貞昌 批鄭麗文情緒性發言對當事人不公,2100273,"[{""start"": 0, ""end"": 142, ""text"": ""針對行政院長蘇貞昌、國...",1,37,2022-02-03T14:55:55.310663Z,2022-02-03T15:02:18.598120Z,518.272


#### 輸入 ckip 帳號密碼

In [4]:
pipeline = CkipPipeline(opts={'con_parser': {'username': '', 'password': ''}})

## 1.代名詞代換

### function 定義

#### 將一字名稱代換
- 呼叫 change_ner_one_word_name( ) 即可  
- ex: `鄭(麗文)` 痛罵 `蘇(貞昌)`

In [5]:
def get_person_entity_list(ner_list):
  # Get the list contain changed person candidates
  person_entity_list = []
  for ner_item in ner_list:
    if ((ner_item[1] in ["PERSON"]) and (2<=len(ner_item[0])<=4) ) or ((ner_item[1] in ["ORG","NORP"]) and (3 <= len(ner_item[0]))): # 
      already_inside = False
      # run through all person in candidates
      for person_item in person_entity_list:
        # have exist in candidate, count add 1
        if ner_item[0] == person_item[0]:
          already_inside = True
          person_item[1] += 1
      # not exist in candidate, append it
      if already_inside == False:
        person_entity_list.append([ner_item[0],0])
  return person_entity_list

In [6]:
def remove_same_first_name_person(person_entity_list):
  # calculate the show count, in order to change person candidate place having same first name
  for item1 in person_entity_list:
    for item2 in person_entity_list:
      # if item not the same, but have same first name
      # move the less show up one to last
      if item1 != item2 and item1[0][0] == item2[0][0]:
        # move less one to last
        if item1[1] < item2[1]:
          person_entity_list.append(person_entity_list.pop(person_entity_list.index(item1)))
        elif item1[1] > item2[1]:
          person_entity_list.append(person_entity_list.pop(person_entity_list.index(item2)))
        elif len(item1[0]) > len(item2[0]):
          person_entity_list.append(person_entity_list.pop(person_entity_list.index(item1)))
        else:
          person_entity_list.append(person_entity_list.pop(person_entity_list.index(item2)))
  return person_entity_list

In [7]:
def change_ner_one_word_name(input_document):
  input_document = input_document.replace(" ","")
  doc = CkipDocument(raw=input_document)
  # Named-Entity Recognition
  pipeline.get_ner(doc)
  ner_list = doc.ner.to_list()
  # change 2d to 1d array
  ner_list = list(chain.from_iterable(ner_list))
  # temp save original ner for later to return
  original_ner_list = [ner[0] for ner in ner_list.copy() if ner[1] not in ['DATE','CARDINAL','PERCENT','QUANTITY','TIME','ORDINAL']]
  
  # call "person entity function"
  person_entity_list = get_person_entity_list(ner_list)
  # call "remove same first name person" function
  person_entity_list = remove_same_first_name_person(person_entity_list)
  # leave person name
  person_entity_list = list([item[0] for item in person_entity_list])

  # sort the place of ner list
  ner_list.sort(key=lambda x: x[2][0], reverse=False)
  # leave the PERSON ner
  ner_list = [[ner_item[0],ner_item[1],(ner_item[2][0],ner_item[2][1])] for ner_item in ner_list if ner_item[1]=='PERSON']

  # change one word name to person entity name
  for i in range(len(ner_list)):
    if ner_list[i][0] not in person_entity_list and ner_list[i][1]=='PERSON' and len(ner_list[i][0]) < 3:
      for person_entity_item in person_entity_list:
        # first name is same, need change
        if ner_list[i][0][0] == person_entity_item[0]:
          new_word_len = len(person_entity_item)
          old_word_len = len(ner_list[i][0])
          input_document = input_document[:ner_list[i][2][0]] + person_entity_item + input_document[ner_list[i][2][1]:]
          for j in range(i+1,len(ner_list)):
            ner_list[j][2] = (ner_list[j][2][0] + (new_word_len-old_word_len), ner_list[j][2][1] + (new_word_len-old_word_len))
  return person_entity_list,input_document,original_ner_list

## 5. 開始實作

### 1.

In [8]:
input_document_list = news_df['article'].tolist()
document_coref_list = []
people_name_list = []
article_ner_list = []
for i in trange(len(input_document_list)):
    # if i == 5:
    #     break
    try:
        article_person_name,document_coref,article_ner = change_ner_one_word_name(input_document_list[i])
        people_name_list.append(article_person_name)
        document_coref_list.append(document_coref)
        article_ner_list.append(article_ner)
    # print(document_coref)
    except:
        people_name_list.append("")
        document_coref_list.append("")
        article_ner_list.append("")
  # if i == stop_point:
  #   break

  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
  1%|          | 1/100 [00:30<50:41, 30.73s/it]

['台灣', '台灣', '台灣', '蔡英文', '國民黨', '立法院', '台灣', '蔡英文', '蘇貞昌', '蔡英文', '台灣', '蘇貞昌', '台灣', '台灣', '中國', '蔡英文', '民進黨', '行政院長', '立法院', '蔡英文', '蔡英文', '台灣人民', '民進黨', '台灣', '台灣', '中常會', '中常會', '蘇貞昌', '蘇貞昌', '台灣', '民進黨', '蔡英文', '鄭麗文']


  2%|▏         | 2/100 [00:34<24:43, 15.14s/it]

['民進黨', '國民黨立院黨團', '袂見', '立法院', '鄭麗文', '李德維', '國民黨團', '蘇貞昌', '蘇貞昌', '鄭麗文', '蔡', '蔡英文', '蘇貞昌', '蘇', '蘇貞昌', '國民黨', '國會', '蘇失格', '蔡英文', '陳明文', '台灣', '行政院長', '蘇貞昌', '國民黨', '蘇貞昌', '國民黨', '蘇貞昌', '蘇', '蔡英文', '蘇貞昌', '鄭先', '國民黨', '李德維', '蘇貞昌', '民進黨', '蘇', '國民黨', '鄭麗文', '野百合學運', '民進黨', '鄭麗文', '黨團', '蔡英文', '蘇', '袂見笑', '費鴻泰酸', '民進黨', '民進黨', '民進黨', '蘇貞昌', '立法院', '國會', '鄭麗文', '國民黨', '鄭麗文', '蘇貞昌', '陳明文', '費鴻泰']


  3%|▎         | 3/100 [00:36<14:31,  8.98s/it]

['蘇貞昌', '中國科興', '國民黨團', '立法院', '國民黨立院黨團', '北檢', '莫德納', '台灣', '嬌生', '蔡其昌', '蔡其昌', '陳玉珍', '蔡其昌', '蔡其昌', '陳柏惟', '國民黨團', 'BNT', '國民黨團', '國民黨團', '國民黨團', '科興', '國民黨團', '費鴻泰', '陳柏惟', 'AZ', '國民黨團', '中國', '北檢', '立法院', '蔡其昌']


  4%|▍         | 4/100 [00:43<12:44,  7.96s/it]

['林俊憲', '高虹安', '日本', 'Nintendo', '高虹安', '金', 'Microsoft', '台灣', '台灣', '紫布', '丁特', '民進黨', '立法院', '高虹安', '邱顯智', '韓國', '洪孟楷', '民眾黨', '虹安', '丁特', '美國', '美國', '高虹安', '時代力量', '台灣', '國民黨', '日本', '虹安', '美國', '韓國', 'Sony', '韓國', 'Nexon', '主丁特', '高虹安', '日本', '高虹安', '高虹安', '9億韓元', '丁特']


  5%|▌         | 5/100 [00:47<14:55,  9.43s/it]

['邱國正', '陳以信', '邱國正', '邱國正', '國防部', '韓岡明', '全民防衛動員署', '國防部', '韓岡明', '國民黨', '國防部全民防衛動員署組織法', '陳以信', '邱國正', '全民防衛動員署', '國防部長', '陳以信', '邱國正', '國防部全民防衛動員綜合作業室', '邱國正', '邱國正', '國防委員會', '全民防衛動員署', '韓岡明', '陳以信', '邱國正', '韓岡明', '陳以信', '立法院', '全民防衛動員署', '韓岡明']


In [9]:
article_coref_df = pd.Series(document_coref_list)
news_df['article_coref'] = article_coref_df

people_name_df = pd.Series(people_name_list)
news_df['people_name'] = people_name_df

article_ner_df = pd.Series(article_ner_list)
news_df['ner'] = article_ner_df

In [10]:
news_df.to_csv(all_output_file, index=False)

In [11]:
news_df.tail(1)

,id,article,category,company_id,reporter,timestamp,title,url_pattern,label,annotator,annotation_id,created_at,updated_at,lead_time,article_coref,people_name,ner
99,855,"行政院長蘇貞昌昨(12日)在立法院會備詢時,用台語怒罵國民黨立委鄭麗文「不要臉」,引發藍營怒...",政治,3,楊蕓,1634091120,張亞中批蘇貞昌嗆鄭麗文不要臉 「歷任最失格的行政院長」,2100005,"[{""start"": 0, ""end"": 48, ""text"": ""行政院長蘇貞昌昨(12日...",1,9,2022-02-03T05:27:35.471495Z,2022-02-03T05:27:35.471520Z,303.946,"行政院長蘇貞昌昨(12日)在立法院會備詢時,用台語怒罵國民黨立委鄭麗文「不要臉」,引發藍營怒...","[朱子, 行政院長, 國民黨, 蘇貞昌, 張亞中, 蔡英文, 民進黨, 立法院, 孫文學校總...","[朱子, 行政院長, 國民黨, 立法院會, 蘇貞昌, 行政院, 張亞中, 蔡英文, 民進黨,..."
